# **Importing Necessary Libraries**

In [1]:
import cv2
from google.colab.patches import cv2_imshow
import sys
import numpy as np

# **Shape Detector**

In [2]:
def shapeDetector (ref_paper, video, resize):
  '''
  Detects and determines the size and shape of the object in the video.
  Also returns the modified image

  Parameters:
    'ref_paper' : 'A3', 'A4' or 'Letter'
      size of the reference paper 
    'video': str
      path to the video
    'resize': tuple of integers (Width, Height)
      resizes each frame to specified values. In case, resizing is not required then set this paramter to None

  ''' 
  #set dim of the reference paper
  if ref_paper == 'A3':
    width = 297 
    height = 420
  elif ref_paper == 'A4':
    width = 210
    height = 297
  elif ref_paper == 'Letter':
    width = 215.9 
    height = 279.4
  else:
    sys.exit("Invalid Value provided for reference paper")

  print('The set dim of the reference paper is (WxH): (', width, 'mm X ', height, 'mm )')
  
  flag = True
  
  #for reading frames in video
  cap = cv2.VideoCapture(video)

  if (cap.isOpened()== False):
    sys.exit("Error opening video stream or file")


  #for storing video
  out = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 30, resize)

  while(True):

    ret, frame = cap.read()

    if ret == True:
      
      #Resizing
      frame =cv2.resize(frame, resize)
      
      #convert to grayscale
      img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

      #Apply gaussian blur
      img = cv2.GaussianBlur(img, (7, 7), 0)

      #Find edges using edge detector
      img = cv2.Canny(img, 50, 100)

      #Perform erosion and dilataion
      img = cv2.dilate(img, None, iterations=1)
      img = cv2.erode(img, None, iterations=1)
      
      #Find Contours

      contour,_ = cv2.findContours(img, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

      orig = frame.copy()
      
      ppm = 0

      for c in reversed(contour):
        
        #Find min Rectangle coordinates around the contour
        rect = cv2.minAreaRect(c)
        box = cv2.boxPoints(rect)
        box = np.array(box, dtype="int")
        
        #Calculate perimeter of bounding box
        peri = cv2.arcLength(box,True)

        #ignore conoturs with perimeter less than threshold
        if peri < 150:
          continue
        
        #Calculate length of each side of bounding box of shape
        temp = box[[3,0,1,2]]
        dist = np.sqrt(np.sum(np.square(box - temp), axis=1, keepdims=True))
      
        #Bounding Box for Reference Paper
        if (peri > 2000):

          #For mapping between mm and pixels
          
          #if vertex of paper is right-bottom
          if abs(box [0][0]-box[0][1]) < (box[0,1]/2):

            w= ((dist[1]+dist[3])/2) / width
            l= ((dist[0]+dist[2])/2) / height
          
            midpoint = (temp + box)//2 
            midpoint [[1,2]] = midpoint [[2,1]]

            po = box[1]

          #if vertex of paper  is left bottom  
          else:
            w= ((dist[0]+dist[2])/2) / width
            l= ((dist[1]+dist[3])/2) / height
          
            midpoint = (temp + box)//2 
            po = box[0]
          
          ppm = (l + w) / 2 #using average values for pixels per mm

          #Draw Contours and display associated properties
          cv2.drawContours(orig, [box], -1, (0, 255, 0), 3)
          cv2.putText(orig, "{:.1f}mm".format(height), tuple(midpoint[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0,0, 255), 2)
          cv2.putText(orig, "{:.1f}mm".format(width), tuple(midpoint[2]), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 0, 255), 2)
          cv2.putText(orig, "Reference Paper: {}".format(ref_paper), tuple(po-10), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 0, 255), 2)
          
          flag =True

        else:

          #Check whether reference page is detected
          if not flag:
            sys.exit("Reference Page not detected")
          
          #Find approx figure
          epsilon = 0.003 * peri
          approx = cv2.approxPolyDP(c, epsilon, True)
          
          #Calculate moments
          mc =cv2.moments(box)
          mc2=cv2.moments(approx)
          
          #Calculate centroids
          mt = np.array((mc['m10'] / (mc['m00'] + 1e-5), mc['m01'] / (mc['m00'] + 1e-5)))
          mt2 = np.array((mc2['m10'] / (mc2['m00'] + 1e-5), mc2['m01'] / (mc2['m00'] + 1e-5)))

          center = ((mt + mt2)//2).astype("int")

          cv2.drawContours(orig, [approx], -1, (255,0,0), 2)

          #if vertex of paper is right-bottom
          if abs(box [0][0]-box[0][1]) < (box[0,1]/2):

            w= ((dist[1]+dist[3])/2) 
            l= ((dist[0]+dist[2])/2) 
          
          #if vertex of paper  is left bottom  
          else:
            w= ((dist[0]+dist[2])/2) 
            l= ((dist[1]+dist[3])/2)     

          
          #Check For Square and Rectangle
          if abs(cv2.arcLength(box, True) - cv2.arcLength(approx, True)) <20:
            
            #For Square
            if abs(l-w) < 10:
              
              cv2.putText(orig, "Square", tuple(center), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
              cv2.putText(orig, "A: {:.1f}mm^2".format(float((l*l)/((ppm*ppm)+ 1e-5))), tuple(center+15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
              cv2.putText(orig, "L: {:.1f}mm".format(float(l/(ppm + 1e-5))), tuple(center+30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

            #For Rectangle
            else:

              cv2.putText(orig, "Rectangle", tuple(center), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
              cv2.putText(orig, "A: {:.1f}mm^2".format(float((l*w)/((ppm*ppm)+ 1e-5))), tuple(center+15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
              cv2.putText(orig, "L: {:.1f}mm".format(float(l/(ppm+ 1e-5))), tuple(center+30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
              cv2.putText(orig, "W: {:.1f}mm".format(float(w/(ppm+ 1e-5))), tuple(center+45), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

          #Else Circle and Triangle 
          else:
            r = ((l+w)/2)/2 #using average value for radius

            ratio =abs(cv2.contourArea(approx, True))/(cv2.arcLength(approx, True)+ 1e-5)
            if ((ratio - (r/2)) < 5) and ((ratio - (r/2)) > -5):
              
              cv2.putText(orig, "Circle",tuple(center), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
              cv2.putText(orig, "A: {:.1f}mm^2".format(float(np.pi*np.square(r)/((ppm*ppm)+ 1e-5))),tuple(center+15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
              cv2.putText(orig, "R: {:.1f}mm".format(float(r/(ppm + 1e-5))), tuple(center+30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

            else:
              cv2.putText(orig, "Triangle", tuple(center), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
              cv2.putText(orig, "(A: {:.1f}mm^2)".format(float((l*w)/((2*ppm*ppm)+ 1e-5))), tuple(center+15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)


      # Write the frame into the file 'output.avi'
      out.write(orig)
        
    else:
      break 

  # release the video capture and video write objects
  cap.release()
  out.release()
  return orig

# **Output**

In [3]:
_ = shapeDetector('A4', '/content/input.mov', (540,908))

The set dim of the reference paper is (WxH): ( 210 mm X  297 mm )
